# Tarefa 2: Usar o SageMaker Clarify
	
Neste laboratório, você vai usar o Amazon SageMaker Clarify para detectar viés nos dados antes do treinamento e nos modelos após o treinamento, além de acessar relatórios de explicabilidade.


## Tarefa 2.1: Configuração do ambiente

Instale os pacotes e as dependências.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sagemaker_datawrangler
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker import clarify
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.s3 import S3Uploader
from sklearn.model_selection import train_test_split
from time import gmtime, strftime

s3_client = boto3.client("s3")
session = Session()
bucket = session.default_bucket()
prefix = 'sagemaker/lab_8'
role = sagemaker.get_execution_role()

Depois, importe, divida e carregue o conjunto de dados.

In [ ]:
#prepare-dataset

lab_test_data = pd.read_csv('adult_data_processed.csv')

# Split the dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)
test_data.to_csv('test_data.csv', index=False, header=False)

# Upload the Dataset to S3
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))
test_path = S3Uploader.upload('test_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')
test_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

Agora, treine o modelo do XGBoost. Use este modelo treinado para o ModelConfig do SageMaker Clarify. Este treinamento exige cerca de três a quatro minutos para ser concluído.

In [ ]:
#train-model

# Retrieve the container image
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework='xgboost', 
    version='1.5-1'
)
# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session
)
# Set the hyperparameters
xgb.set_hyperparameters(
    max_depth=5, 
    eta=0.2, 
    gamma=4, 
    min_child_weight=6,
    subsample=0.8, 
    verbosity=1, 
    objective='binary:logistic', 
    num_round=800
)

# Train the model
xgb.fit(
    inputs = data_inputs
) 

## Tarefa 2.2: Configurar o trabalho do SageMaker Clarify e acessar o relatório de viés

Para usar o [SageMaker Clarify](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-configure-processing-jobs.html), primeiro você cria um modelo por meio do trabalho de treinamento. Depois, você define as configurações necessárias e executa o trabalho SageMaker Clarify no modelo treinado. Nesta tarefa, você vai:

- Criar o modelo.
- Ativar o SageMaker Clarify.
- Gerar o relatório de viés.
- Acessar o relatório.

### Tarefa 2.2.1: criar o modelo

Crie um modelo por meio do trabalho de treinamento a ser usado com o SageMaker Clarify.

In [ ]:
#create-clarify-model

model_name = "lab-8-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
session.create_model(model_name, role, container_def)

### Tarefa 2.2.2 Ativar o SageMaker Clarify

Agora, para iniciar a configuração, ative o SageMaker Clarify.

In [ ]:
#enable-clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=session
)

Com o SageMaker Clarify, você pode detectar um possível viés antes do treinamento nos dados e um viés após o treinamento nos modelos por meio de várias métricas. Consulte [métricas](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) para obter mais informações de como configurar a análise do Clarify.

Configure o objeto **DataConfig** para comunicar informações de E/S de dados ao SageMaker Clarify. Aqui, você especifica o local do conjunto de dados de entrada, o local da saída, a coluna de entrada (**label**), os nomes de cabeçalho e o tipo de conjunto de dados.

In [ ]:
#define-data-config

bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="income",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

Configure um objeto **ModelConfig** para comunicar informações sobre o modelo treinado. Defina o nome do modelo e um endpoint dedicado temporário para o modelo de produção (**instance_type** e **instance_count**). Defina também um **accept_type** para indicar o formato do payload de resposta do endpoint e um **content_type** para indicar o formato do payload de solicitação ao endpoint.

In [ ]:
#define-model-config

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

Configure o **probability_threshold** de **ModelPredictedLabelConfig** para converter a probabilidade de amostras em rótulos binários visando a análise de viés. Uma previsão superior ao limite é interpretada como o valor de rótulo **1** e, igual ou inferior ao limite, como o valor de rótulo 0.

In [ ]:
#define-model-predicted-label-config

predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

Configure **BiasConfig** para indicar as colunas sensíveis (**facets**), os recursos possivelmente sensíveis (**facet_values_or_threshold**) e os resultados favoráveis (**label_values_or_threshold**).

Você pode indicar dados categóricos e contínuos para **facet_values_or_threshold** e **label_values_or_threshold**. Os recursos **sex** e **age** são categóricos.

A meta desse modelo é determinar se uma pessoa ganha USD 50 mil ou mais, com o resultado positivo considerado favorável. Use **BiasConfig** para saber quais colunas contêm as facetas com o grupo sensível **Sex**, o que os recursos sensíveis podem estar usando **facet_values_or_threshold** e o que os resultados desejados estão usando **label_values_or_threshold**. Agrupe por **age** para ver se há diferenças dependendo da idade da pessoa.

In [ ]:
#define-bias-config

bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="sex", facet_values_or_threshold=[0], group_name="age"
)

### Tarefa 2.2.3: gerar o relatório de viés

Crie o relatório de viés usando as configurações para análise antes e após o treinamento. Esta etapa exige cerca de 15 a 20 minutos. Enquanto o relatório de viés está sendo gerado, você pode descobrir como o SageMaker Clarify faz os cálculos [Medidas de equidade para machine learning em finanças](./Fairness.Measures.for.Machine.Learning.in.Finance.pdf).

In [ ]:
#run-bias-report

clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

### Tarefa 2.2.4: Acessar o relatório de viés

No SageMaker Studio, você pode visualizar os resultados na aba de experimentos.

A próxima etapa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **lab_8.ipynb** para a lateral ou escolha (clique com o botão direito) a aba **lab_8.ipynb** e selecione **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora o relatório de viés.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar o relatório de viés, retorne ao notebook selecionando a aba **lab_8.ipynb**.

1. No SageMaker Studio, selecione o ícone **Página inicial do SageMaker**.

2. Escolha **Experiments** (Experimentos).

O SageMaker Studio abre a aba **Experiments** (Experimentos).

3. Na aba **Experiments** (Experimentos), selecione **Unassigned runs** (Execuções não atribuídas) à esquerda.

4. Na lista **Unassigned runs** (Execuções não atribuídas), selecione o nome do trabalho de treinamento que contém **clarify-bias-** no título. 

5. Na aba **Experiments** (Experimentos), selecione **Bias Reports** (Relatórios de viés) à esquerda.

Um relatório de viés das métricas é disponibilizado quando o SageMaker Clarify termina a execução do trabalho de viés.

6. Espere até que o SageMaker Clarify termine a execução do trabalho de viés.

Cada métrica de viés tem explicações detalhadas com exemplos que você pode explorar ao selecionar cada valor. 

7. Examine as explicações detalhadas de **Class Imbalance** (Desequilíbrio de classe) e **Disparate (Adverse) Impact (DI)** (Impacto (adverso) da discrepância (DI)) clicando nas setas ao lado das métricas de viés e expandindo o campo. 

Quando terminar de explorar as métricas de viés, continue na próxima tarefa.

## Tarefa 2.3: Acessar relatórios de explicabilidade

Além de usar um relatório de viés, você pode usar o SageMaker Clarify para analisar as explicações locais de previsões individuais. Crie um relatório para examinar os resultados de explicabilidade das previsões geradas pelo SageMaker Clarify e analise as métricas principais do relatório. Nesta tarefa, você vai:

1. Defina uma configuração de valores Shapley (SHAP).
2. Gerar o relatório de explicabilidade.
3. Acessar o relatório de explicabilidade.

### Tarefa 2.3.1: Definir uma configuração de SHAP

Você pode gerar um relatório de explicabilidade para obter uma explicação do motivo pelo qual o modelo fez uma previsão específica. O SageMaker Clarify usa o SHAP para gerar um relatório sobre o raciocínio do modelo para tomar decisões. Consulte [Linhas de base do SHAP para explicabilidade](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-feature-attribute-shap-baselines.html) para saber mais sobre as linhas de base de SHAP.

As métricas do SHAP configuradas são as seguintes:
- **baseline**: uma lista linhas ou um URI de objeto do Amazon Simple Storage Service (Amazon S3), a ser usada como o conjunto de dados de linha de base no algoritmo do SHAP do kernel.
- **num_samples**: número de exemplos a serem usados no algoritmo do SHAP do kernel. Esse número determina o tamanho do conjunto de dados sintético gerado para calcular os valores SHAP.
- **agg_method: mean_abs**: média de valores SHAP absolutos para todas as instâncias.
- **save_local_shap_values**: valor booliano para indicar se os valores SHAP locais devem ser salvos no local de saída.

Consulte [métricas do SHAP](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-processing-job-configure-analysis.html) para obter mais informações sobre as métricas usadas.

In [ ]:
#configure-shap

testing_data, clarify_data = train_test_split(test_data, test_size =0.005)
clarify_data = clarify_data.drop(columns=["income"])
clarify_data.to_csv('clarify_data.csv', index=False, header=False)
clarify_path = S3Uploader.upload('clarify_data.csv', 's3://{}/{}'.format(bucket, prefix))

shap_config = clarify.SHAPConfig(
    baseline=clarify_path,
    num_samples=15,
    agg_method="mean_abs",
    save_local_shap_values=True,
)

explainability_output_path = "s3://{}/{}/clarify-explainability".format(bucket, prefix)
explainability_data_config = clarify.DataConfig(
    s3_data_input_path=clarify_path,
    s3_output_path=explainability_output_path,
    headers=clarify_data.columns.to_list(),
    dataset_type="text/csv",
)

### Tarefa 2.3.2: Executar o relatório de explicabilidade

Crie o relatório de explicabilidade usando suas configurações. Esta etapa exige cerca de 10 a 15 minutos. Enquanto o relatório de explicabilidade está sendo gerado, você pode acompanhar o status do trabalho no SageMaker Studio continuando na próxima tarefa.

Consulte [Whitepaper de equidade e explicabilidade de IA da Amazon](./Amazon.AI.Fairness.and.Explainability.Whitepaper.pdf) para obter mais informações sobre o processo de explicabilidade do SageMaker Clarify.

In [ ]:
#run-explainability-report

clarify_processor.run_explainability(
    data_config=explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)

### Tarefa 2.3.3: Acessar o relatório de explicabilidade

Assim como no relatório de viés, você pode visualizar o relatório de explicabilidade no Studio como parte dos experimentos.

1. Clique na aba **Experiments** (Experimentos).

2. Na aba **Experiments** (Experimentos), selecione **Unassigned runs** (Execuções não atribuídas) à esquerda.

3. Na lista **Unassigned runs** (Execuções não atribuídas), selecione o nome do trabalho de treinamento que contém **clarify-bias-** no título. 

4. Na aba **Experiments** (Experimentos), selecione **Explainability** (Explicabilidade) à esquerda.

5. Quando o trabalho de explicabilidade for concluído, para visualizar o gráfico **Feature Importance** (Importância do recurso), selecione <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">View sample notebook (Visualizar notebook de exemplo)</span>.

O SageMaker Studio abre o notebook **Fairness and Explainability with SageMaker Clarify** (Equidade e explicabilidade com o SageMaker Clarify) em uma aba.

Quais recursos têm a maior e a menor importância? Há resultados surpreendentes?

### Conclusão

Parabéns! Você usou o SageMaker Clarify para criar relatórios de viés e explicabilidade que podem ser usados para desenvolver modelos mais confiáveis e em maior conformidade. No próximo laboratório, você vai implantar os modelos e executar a inferência. Você vai continuar trabalhando com o modelo no próximo laboratório.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.